In [12]:
import os

In [13]:
%pwd

'c:\\Users\\Desktop\\xReayDetection\\research'

In [14]:
os.chdir('../')

In [15]:
%pwd

'c:\\Users\\Desktop\\xReayDetection'

In [16]:

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [17]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/SRVHCCC/kedney-desease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [19]:
import tensorflow as tf
from pathlib import Path
import dagshub
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [28]:
class Evaluation:
    def __init__(self, config):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        # Ensure the model and valid_generator are available
        if self.model and self.valid_generator:
            self.score = self.model.evaluate(self.valid_generator)
            self.save_score()
        else:
            raise ValueError("Model or validation generator could not be loaded properly.")

    def save_score(self):
        # Ensure score has valid values
        if self.score and len(self.score) >= 2:
            scores = {"loss": self.score[0], "accuracy": self.score[1]}
            self.save_json(path=Path("scores.json"), data=scores)
        else:
            raise ValueError("Score should have at least loss and accuracy.")

    @staticmethod
    def save_json(path: Path, data: dict):
        # Saves the data into a JSON file
        with open(path, "w") as json_file:
            json.dump(data, json_file)

    def log_into_mlflow(self):
        try:
            # Set the tracking URI and validate
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            # Start an MLflow run
            with mlflow.start_run():
                # Log parameters
                if isinstance(self.config.all_params, dict):
                    mlflow.log_params(self.config.all_params)
                else:
                    raise ValueError("self.config.all_params must be a dictionary")

                # Log metrics (ensure score has loss and accuracy)
                if len(self.score) >= 2:
                    mlflow.log_metrics(
                        {"loss": self.score[0], "accuracy": self.score[1]}
                    )
                else:
                    raise ValueError("self.score should have at least 2 values: [loss, accuracy]")

                # Log and register the model
                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
        
        except Exception as e:
            print(f"Error logging into MLflow: {e}")
            raise

In [29]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Directory created or already exists: artifacts
Found 3732 images belonging to 4 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\hp\anaconda3\envs\cnncls\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\hp\a